In [1]:
from src import clean_data, model, training

import polars as pl
import torch

user_matrix, movies = clean_data()

test = user_matrix.sample(fraction=0.1).to_pandas()
train = user_matrix.filter(~pl.col("user").is_in(test["user"]))

model = model.AutoEncoder(len(movies), n_latent_factors=32).to("cuda")
model = torch.load("model.pt")
optim = torch.optim.Adam(model.parameters(), lr=0.001)

Cleaning data...
Data cleaned!


In [25]:
import torch
from tqdm import tqdm

pbar = tqdm(range(100_000))

for batch_number in pbar:        
    loss = training.train_step(
        model,
        optim,
        *training.df_sample_to_batch(train.sample(2048), len(movies), "cuda")
    ).item()
    pbar.set_description(f"loss is: {int(loss)}")

loss is: 870: 100%|██████████| 100000/100000 [58:46<00:00, 28.35it/s]


In [20]:
harry_potter_vector = torch.zeros(len(movies)).to("cuda")
harry_potter_vector[628] = 2
harry_potter_vector[945] = 2
harry_potter_vector[994] = 2
harry_potter_vector[1120] = 2
harry_potter_vector[1860] = 2
harry_potter_vector[1097] = 2

model.eval()
predictions = model(harry_potter_vector.unsqueeze(0)).squeeze().cpu().detach().numpy()
model.train()
movies_pandas = movies.to_pandas()
movies_pandas[movies_pandas["new_id"].isin(predictions.argsort()[-20:][::-1])]

,film,year,title,new_id
43,361,2004,The Phantom of the Opera: Special Edition,43
200,1902,1979,Star Trek: The Motion Picture,200
437,3960,1960,Pollyanna,437
618,5519,1999,The West Wing: Season 1,618
628,5582,1980,Star Wars: Episode V: The Empire Strikes Back,628
637,5646,2001,61*,637
640,5656,1986,Star Trek IV: The Voyage Home,640
719,6268,1998,Lost in Space,719
945,8292,2004,Star Wars: Clone Wars: Vol. 1,945
994,8687,2002,Star Wars: Episode II: Attack of the Clones,994


In [17]:
movies_pandas[movies_pandas["title"].str.contains("star wars", case=False)]

,film,year,title,new_id
628,5582,1980,Star Wars: Episode V: The Empire Strikes Back,628
945,8292,2004,Star Wars: Clone Wars: Vol. 1,945
994,8687,2002,Star Wars: Episode II: Attack of the Clones,994
1097,9628,1983,Star Wars: Episode VI: Return of the Jedi,1097
1120,9886,1999,Star Wars: Episode I: The Phantom Menace,1120
1860,16265,1977,Star Wars: Episode IV: A New Hope,1860
